In [7]:
import requests
from bs4 import BeautifulSoup

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

user_message = [{
    "role":"user",
    "content":"Hello! This is my first ever message to you!"
}]

payload = {
    "model": MODEL,
    "messages": user_message,
    "stream": False
}

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Welcome! I'm thrilled to be your conversational partner for the first time. I'm here to help answer any questions, provide information, or simply chat with you about anything that's on your mind. How can I assist you today? Is there something specific you'd like to talk about, or do you want to get to know me better?


In [11]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class WebsiteCrawler:
    def __init__(self, url):
        self.url = url
        resp = requests.get(url, headers=headers)
        self.body = resp.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            else:
                self.text = ""
            links = [link.get('href') for link in soup.find_all('a')]
            self.links = [link for link in links if link]
            
    def get_contents(self):
        return f"Webpage Title: {self.title}\nWebpage URL: {self.url}\nWebpage Content: {self.text}\n\n"


In [31]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
take only link of About page.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
    ]
}
"""

print(link_system_prompt)

def get_links_user_prompt(website):
    user_prompt = f"Here is the about us link on the website of {website.url}"
    return user_prompt

testpage = WebsiteCrawler("https://www.siftandpick.com")
print(get_links_user_prompt(testpage))

You are provided with a list of links found on a webpage. take only link of About page.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
    ]
}

Here is the about us link on the website of https://www.siftandpick.com


In [35]:
import json


def get_links(url):
    global MODEL
    webpage = WebsiteCrawler(url)    
    _payload = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(webpage)}
        ],
        "stream": False
    }
    res = requests.post(OLLAMA_API, json=_payload, headers=HEADERS)
    content = res.json()['message']['content']
    parsed_content = json.loads(content)
    return parsed_content.get("links")[0]["url"]

get_links("https://www.siftandpick.com")

'https://www.siftandpick.com/pages/about'